<a href="https://colab.research.google.com/github/aubreymoore/crb-yona-tiff/blob/main/slicing_tif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from osgeo import gdal

In [2]:
!git clone https://github.com/aubreymoore/crb-yona-tiff.git

Cloning into 'crb-yona-tiff'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 151 (delta 63), reused 17 (delta 4), pack-reused 30
Receiving objects: 100% (151/151), 1.06 GiB | 29.76 MiB/s, done.
Resolving deltas: 100% (63/63), done.
Updating files: 100% (32/32), done.


In [6]:
!cat crb-yona-tiff/chunks/* > yona.tif

In [7]:
!ls

crb-yona-tiff  sample_data  yona.tif


In [9]:
rds = gdal.Open('yona.tif')

In [12]:
print(f'width = {rds.RasterXSize}, height = {rds.RasterYSize}')

width = 32731, height = 23585


In [13]:
rds.GetGeoTransform()

(259821.30138992154,
 0.014080873475957724,
 0.0,
 1484150.4414387643,
 0.0,
 -0.014080873475957724)